# Tensorflow Abstractions - Estimator API

## Import Libraries

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

## Load Data

In [2]:
wine_data = load_wine()

In [3]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [4]:
print(wine_data["DESCR"])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

## Preprocess Data

In [5]:
feat_data = wine_data["data"]

In [6]:
labels = wine_data["target"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.30, random_state=101)

In [8]:
scaler = MinMaxScaler()

In [9]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Estimator API

In [10]:
from tensorflow import estimator

In [11]:
X_train.shape

(124, 13)

In [12]:
X_train

array([[1.187e+01, 4.310e+00, 2.390e+00, ..., 7.500e-01, 3.640e+00,
        3.800e+02],
       [1.217e+01, 1.450e+00, 2.530e+00, ..., 1.450e+00, 2.230e+00,
        3.550e+02],
       [1.234e+01, 2.450e+00, 2.460e+00, ..., 8.000e-01, 3.380e+00,
        4.380e+02],
       ...,
       [1.272e+01, 1.810e+00, 2.200e+00, ..., 1.160e+00, 3.140e+00,
        7.140e+02],
       [1.412e+01, 1.480e+00, 2.320e+00, ..., 1.170e+00, 2.820e+00,
        1.280e+03],
       [1.247e+01, 1.520e+00, 2.200e+00, ..., 1.160e+00, 2.630e+00,
        9.370e+02]])

In [13]:
feat_cols = [tf.feature_column.numeric_column("x", shape=(13))]

In [14]:
deep_model = estimator.DNNClassifier(hidden_units=[13, 13, 13], 
                                     feature_columns=feat_cols, 
                                     n_classes=3, 
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmp1wci28f6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a21ad1438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
input_func = estimator.inputs.numpy_input_fn(x={"x": X_train_scaled},
                                            y=y_train,
                                            shuffle=True,
                                            batch_size=10,
                                            num_epochs=5)

In [16]:
deep_model.train(input_fn=input_func, steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmp1wci28f6/model.ckpt.
INFO:tensorflow:loss = 11.394388, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmp1wci28f6/model.ckpt.
INFO:tensorflow:Loss for final step: 8.156385.


In [17]:
input_func_eval = estimator.inputs.numpy_input_fn(x={"x":X_test_scaled}, 
                                                  shuffle=False)

In [18]:
preds = list(deep_model.predict(input_fn=input_func_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/jf/g8tp5ct56hl6p6r66zl2knzc0000gn/T/tmp1wci28f6/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
predictions = [p["class_ids"][0] for p in preds]

In [20]:
from sklearn.metrics import confusion_matrix, classification_report

In [21]:
print(classification_report(y_test, predictions))
print("\n")
print(confusion_matrix(y_test, predictions))

             precision    recall  f1-score   support

          0       0.68      1.00      0.81        19
          1       0.87      0.91      0.89        22
          2       1.00      0.23      0.38        13

avg / total       0.83      0.78      0.74        54



[[19  0  0]
 [ 2 20  0]
 [ 7  3  3]]
